# Outil d'alignement de données géolocalisées et de récupération d'informations

*Documents d'entrée* : fichiers au format .txt contenant une liste d'entités nommées de lieux.

*Documents de sortie* : un fichier Excel contenant la liste des lieux reconnus avec les ajouts issus de Wikidata + un fichier JSON contenant les mêmes informations (utilisé ensuite pour réaliser les cartographies).


In [ ]:
#@markdown # Connecter le notebook à son compte Google Drive

#@markdown - Lancer la cellule
#@markdown - Cliquer sur « Exécuter malgré tout » lors de l’apparition du message d’avertissement indiquant que le notebook n’a pas été créé par Google
#@markdown - Cliquer sur « Se connecter à Google Drive » lors de l’apparition du second message d’avertissement pour donner l’autorisation au notebook d’accéder à vos fichiers Google Drive
#@markdown - Choisir son compte Gmail puis cliquer sur « Autoriser »

from google.colab import drive
!pip install wptools==0.4.17
import wptools
import glob
from collections import Counter
import re
import os
import json
import urllib.parse, urllib.request
import pandas as pd


drive.mount('/content/drive/')
%cd /content/drive/My Drive/

In [ ]:
#@markdown # Alignement des données :

#@markdown ####Indiquer le chemin absolu vers le dossier de travail sur le Google Drive :
chemin_vers_le_dossier_de_travail = '/content/drive/My Drive/tutoriel_extraction_cartographie-main/'#@param {type:"string"}
os.chdir(chemin_vers_le_dossier_de_travail)

#@markdown ####Indiquer le chemin absolu vers le dossier contenant les entités nommées de lieux :
chemin_vers_les_listes_de_lieux = '/content/drive/MyDrive/tutoriel_extraction_cartographie-main/entites lieux guides/'#@param {type:"string"}

#@markdown ####Indiquer la langue pour procéder à l'alignement :
langue_pour_l_alignement = "Francais" #@param ["Anglais", "Italien", "Francais", "Allemand", "Espagnol"]

#@markdown ####Indiquer le nom du fichier JSON créé :
nom_fichier_donnees_lieux = 'donnees lieux corpus guides.json'#@param {type:"string"}

#@markdown ####Indiquer le nom du fichier Excel créé :
nom_fichier_excel_donnes_lieux = 'donnees lieux corpus guides tab.xlsx'#@param {type:"string"}

if langue_pour_l_alignement == "Espagnol":
  langue_pour_l_alignement = "es"
if langue_pour_l_alignement == "Francais":
  langue_pour_l_alignement = "fr"
if langue_pour_l_alignement == "Anglais":
  langue_pour_l_alignement = "en"
if langue_pour_l_alignement == "Allemand":
  langue_pour_l_alignement = "de"
if langue_pour_l_alignement == "Italien":
  langue_pour_l_alignement = "it"

import pycurl
def CallWikifier(text, lang=langue_pour_l_alignement, threshold=0.9): 
    nom, type_lieu, latitude, longitude, description = ([] for i in range(5))
  # Thresold = taux de sureté. 0.8 = sûr que le résultat est exact; mais peu de résultat.
  # 1 = tous les résultats mais certains faux résultats
    # Url de demande:
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "4"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
    # Appel de Wikifier 
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 100) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
        print(response)
        for annotation in response["annotations"]:
          ## Récupération (uniquement de l'ID wikidata):
          try:
            itemid = annotation["wikiDataItemId"] 
            page = wptools.page(wikibase=itemid, lang=langue_pour_l_alignement)
            page.get_wikidata()
            page.get_parse()
                      ## Pour avoir toutes les informations:

            infobox=page.data['infobox']
            nom = infobox.get("nom")
            type_lieu=infobox.get("type")
            if type_lieu is None: 
                  type_lieu=page.data['what']
            latitude = infobox.get("latitude")
            longitude = infobox.get("longitude")
            description = page.data["description"]
          except (AttributeError, pycurl.error, KeyError) as at:
                print("Ce n'est pas un lieu!")
            # Création d'un dictionnaire + fichier json avec les informations récupérées sur les lieux:
          donnees[str(nom)] = {"Type de lieu" : type_lieu,
                              "Latitude" : str(latitude), "Longitude" : str(longitude), "Description": str(description)}    
        


            # Enregistrement de toutes les informations récupérées sous la forme d'un tableau Excel:
    df = pd.DataFrame(donnees)
    df2 = df.transpose()
    df2.to_excel(nom_fichier_excel_donnes_lieux)
    with open(nom_fichier_donnees_lieux, "w") as fichier : 
        json.dump(donnees, fichier, ensure_ascii=False)

donnees ={}
toutes_listes = ''.join(chemin_vers_les_listes_de_lieux+"/*.txt")
print(toutes_listes)
for recos in glob.glob(toutes_listes):
    with open(recos, "r") as myfile:
      liste=myfile.readlines()
      for lieu in liste:
        print(lieu)
        CallWikifier(text=lieu, lang=langue_pour_l_alignement)


In [ ]:
#@markdown # Calculs statistiques des types de lieux

#@markdown Le résultat s'affiche en dessous de la cellule (aucun fichier est créé par l'exécution de cette cellule).

liste_des_types = []
fichier_json = nom_fichier_donnees_lieux
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_types=value["Type de lieu"]
      liste_des_types.append(test_types)
for ele in liste_des_types:
  if type(ele) == list:
    liste_des_types.remove(ele)

def proportion_types(liste):
  c = Counter(item for item in liste)
  print("Voici le nombre de lieux, triés par chaque type :")
  print(c) ## Affiche le nombre de lieux de chaque type
  prop = [(i, c[i] / len(liste) * 100.0) for i in c]
  print("Voici la proportion de chaque type de lieux :")
  print(prop) ## Affiche la proportion de chaque type de lieux
  
proportion_types(liste_des_types)
